# Chapter 42: TypeScript at Scale

---

## Introduction

As TypeScript applications grow from single repositories to enterprise-grade systems spanning multiple packages, libraries, and services, managing code at scale becomes critical. This chapter explores architectural patterns, tooling, and best practices for maintaining large-scale TypeScript codebases using monorepos—repositories containing multiple related projects with shared dependencies and type definitions.

Whether you're building a microservices architecture, a design system with multiple packages, or a full-stack application with shared types between frontend and backend, this chapter provides the industry-standard approaches used by organizations like Google, Microsoft, Netflix, and Stripe.

---

## 42.1 Monorepo Structure

A monorepo (monolithic repository) is a version-controlled code repository that holds many projects, often with shared dependencies and interdependent code. While polyrepos (one repository per project) isolate concerns, monorepos excel at code sharing, atomic refactoring, and cross-project visibility.

### 42.1.1 Understanding Monorepo Architecture

**Monorepo vs. Polyrepo:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Monorepo vs Polyrepo Comparison                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Monorepo                          │   Polyrepo                    │
│  ───────────────────────────────────┼────────────────────────────── │
│                                     │                               │
│   📁 my-company/                    │   📁 frontend-app/            │
│   ├── 📁 packages/                  │   ├── src/                    │
│   │   ├── 📁 ui-components/         │   └── package.json            │
│   │   ├── 📁 api-client/            │                               │
│   │   ├── 📁 shared-types/          │   📁 backend-api/             │
│   │   └── 📁 utils/                 │   ├── src/                    │
│   ├── 📁 apps/                      │   └── package.json            │
│   │   ├── 📁 web-app/               │                               │
│   │   └── 📁 mobile-app/            │   📁 shared-types/            │
│   └── package.json (workspaces)     │   ├── src/                    │
│                                     │   └── package.json            │
│                                     │                               │
│   ✓ Atomic changes across apps      │   ✓ Independent versioning    │
│   ✓ Shared type safety              │   ✓ Team autonomy             │
│   ✓ Single CI/CD pipeline           │   ✓ Smaller repo sizes        │
│   ✓ Easier refactoring              │   ✓ Different tech stacks     │
│   ✓ Unified tooling                 │   ✓ Clear boundaries          │
│                                     │                               │
│   ✗ Larger repository size          │   ✗ Type duplication          │
│   ✗ Complex build systems           │   ✗ Cross-repo changes        │
│   ✗ Access control challenges       │   ✗ Version synchronization   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 42.1.2 Workspace Configuration

Modern monorepos leverage package manager workspaces (npm, Yarn, or pnpm) to link local packages without publishing to npm.

**npm Workspaces (v7+):**

```typescript
// Root package.json
{
  "name": "@mycompany/monorepo",
  "version": "1.0.0",
  "private": true,
  "workspaces": [
    "packages/*",
    "apps/*"
  ],
  "scripts": {
    "build": "npm run build --workspaces",
    "test": "npm run test --workspaces",
    "lint": "eslint . --ext .ts,.tsx"
  },
  "devDependencies": {
    "typescript": "^5.3.0",
    "@types/node": "^20.0.0",
    "eslint": "^8.0.0",
    "@typescript-eslint/eslint-plugin": "^6.0.0"
  }
}
```

**Explanation:**
- `workspaces`: Defines glob patterns for workspace packages
- `--workspaces` flag: Runs scripts across all workspaces
- Root dependencies: Shared devDependencies installed once at root

**Yarn Workspaces (Classic & Berry):**

```typescript
// Root package.json
{
  "name": "@mycompany/monorepo",
  "private": true,
  "workspaces": [
    "packages/*",
    "apps/*"
  ],
  "packageManager": "yarn@3.6.0"
}

// .yarnrc.yml (for Yarn Berry/Plug'n'Play)
nodeLinker: node-modules
nmHoistingLimits: workspaces

typescript:
  enableGlobalCache: true
```

**pnpm Workspaces (Recommended for TypeScript):**

```typescript
// pnpm-workspace.yaml
packages:
  - 'packages/*'
  - 'apps/*'
  - '!**/test/**'  # Exclude test directories

// Root package.json
{
  "name": "@mycompany/monorepo",
  "private": true,
  "scripts": {
    "build": "pnpm --filter=\"./packages/**\" build",
    "dev": "pnpm --parallel --filter=\"./apps/**\" dev",
    "typecheck": "tsc --build"
  },
  "devDependencies": {
    "@mycompany/tsconfig": "workspace:*",
    "typescript": "^5.3.0"
  }
}
```

**Explanation:**
- pnpm uses content-addressable storage (saves disk space)
- `workspace:*` protocol links local packages
- `--parallel` runs scripts concurrently across workspaces
- Filtering allows targeting specific package subsets

### 42.1.3 TypeScript Project References

For large monorepos, TypeScript's Project References enable incremental builds and clear dependency graphs between packages.

**Configuration Structure:**

```typescript
// packages/shared-types/tsconfig.json
{
  "compilerOptions": {
    "composite": true,        // Required for project references
    "declaration": true,      // Generates .d.ts files
    "declarationMap": true,   // Source maps for declarations
    "outDir": "./dist",
    "rootDir": "./src",
    "strict": true
  },
  "include": ["src/**/*"],
  "exclude": ["dist", "node_modules", "**/*.test.ts"]
}

// packages/api-client/tsconfig.json
{
  "compilerOptions": {
    "composite": true,
    "outDir": "./dist",
    "rootDir": "./src"
  },
  "include": ["src/**/*"],
  "references": [
    { "path": "../shared-types" }  // Depends on shared-types
  ]
}

// Root tsconfig.json (solution file)
{
  "files": [],
  "references": [
    { "path": "./packages/shared-types" },
    { "path": "./packages/api-client" },
    { "path": "./packages/ui-components" },
    { "path": "./apps/web" },
    { "path": "./apps/api" }
  ]
}
```

**Explanation:**
- `composite: true`: Enables project reference constraints (must have `declaration`, `rootDir`, etc.)
- `references`: Declares dependencies between projects
- Solution file: Root tsconfig that references all projects but contains no files
- Incremental builds: TypeScript only rebuilds changed projects and their dependents

**Building with Project References:**

```typescript
// Root package.json scripts
{
  "scripts": {
    "typecheck": "tsc --build",
    "typecheck:watch": "tsc --build --watch",
    "clean": "tsc --build --clean"
  }
}

// Building specific package and dependencies
// tsc --build packages/api-client --force
```

---

## 42.2 Nx for TypeScript Monorepos

Nx is a comprehensive build system and monorepo tool developed by Nrwl, designed specifically for TypeScript/Node.js ecosystems. It provides intelligent caching, code generation, and dependency graph analysis.

### 42.2.1 Setting Up Nx

**Installation and Configuration:**

```typescript
// Initialize Nx workspace (using npx)
// npx create-nx-workspace@latest myorg --preset=ts

// nx.json - Nx configuration
{
  "extends": "nx/presets/npm.json",
  "npmScope": "myorg",
  "tasksRunnerOptions": {
    "default": {
      "runner": "nx/tasks-runners/default",
      "options": {
        "cacheableOperations": ["build", "lint", "test", "e2e"]
      }
    }
  },
  "targetDefaults": {
    "build": {
      "dependsOn": ["^build"],
      "inputs": ["production", "^production"],
      "outputs": ["{projectRoot}/dist"]
    }
  },
  "namedInputs": {
    "default": ["{projectRoot}/**/*", "sharedGlobals"],
    "production": [
      "default",
      "!{projectRoot}/**/*.spec.ts",
      "!{projectRoot}/**/*.test.ts"
    ]
  }
}
```

**Explanation:**
- `npmScope`: Prefix for all packages (e.g., `@myorg/shared-types`)
- `cacheableOperations`: Tasks Nx will cache based on input hashes
- `dependsOn`: `^build` means build dependencies first (upstream builds)
- `namedInputs`: Reusable filesets for cache calculation

### 42.2.2 Nx Generators and Executors

Nx provides code generators for consistent package creation and executors for standardized build tasks.

**Project Configuration (project.json):**

```typescript
// packages/api-client/project.json
{
  "name": "api-client",
  "$schema": "../../node_modules/nx/schemas/project-schema.json",
  "sourceRoot": "packages/api-client/src",
  "projectType": "library",
  "targets": {
    "build": {
      "executor": "@nx/js:tsc",
      "outputs": ["{options.outputPath}"],
      "options": {
        "outputPath": "packages/api-client/dist",
        "main": "packages/api-client/src/index.ts",
        "tsConfig": "packages/api-client/tsconfig.lib.json",
        "assets": ["packages/api-client/*.md"]
      }
    },
    "lint": {
      "executor": "@nx/linter:eslint",
      "outputs": ["{options.outputFile}"],
      "options": {
        "lintFilePatterns": ["packages/api-client/**/*.ts"]
      }
    },
    "test": {
      "executor": "@nx/jest:jest",
      "outputs": ["{workspaceRoot}/coverage/{projectName}"],
      "options": {
        "jestConfig": "packages/api-client/jest.config.ts",
        "passWithNoTests": true
      }
    }
  },
  "tags": ["scope:shared", "type:util"]
}
```

**Explanation:**
- `executor`: Pre-configured build tools (tsc, webpack, vite, etc.)
- `outputs`: Cacheable output paths for incremental builds
- `tags`: Enforce module boundaries (e.g., `scope:frontend` can't import `scope:backend`)

### 42.2.3 Nx Dependency Graph and Affected Commands

Nx analyzes the dependency graph to only rebuild/test what changed.

**Dependency Graph Commands:**

```typescript
// Visualize dependency graph
// npx nx graph

// Run tests only for affected projects
// npx nx affected:test --base=main --head=HEAD

// Build only affected libraries and their dependents
// npx nx affected:build --base=main~1

// Lint specific project
// npx nx lint api-client

// Run target for specific project
// npx nx build api-client
```

**Module Boundary Enforcement:**

```typescript
// .eslintrc.json - Enforce architectural boundaries
{
  "root": true,
  "plugins": ["@nx"],
  "overrides": [
    {
      "files": ["*.ts", "*.tsx", "*.js", "*.jsx"],
      "rules": {
        "@nx/enforce-module-boundaries": [
          "error",
          {
            "enforceBuildableLibDependency": true,
            "allow": [],
            "depConstraints": [
              {
                "sourceTag": "scope:shared",
                "onlyDependOnLibsWithTags": ["scope:shared"]
              },
              {
                "sourceTag": "scope:frontend",
                "onlyDependOnLibsWithTags": ["scope:shared", "scope:frontend"]
              },
              {
                "sourceTag": "scope:backend",
                "onlyDependOnLibsWithTags": ["scope:shared", "scope:backend"]
              },
              {
                "sourceTag": "type:feature",
                "onlyDependOnLibsWithTags": ["type:feature", "type:ui", "type:util", "type:data"]
              },
              {
                "sourceTag": "type:ui",
                "onlyDependOnLibsWithTags": ["type:ui", "type:util"]
              }
            ]
          }
        ]
      }
    }
  ]
}
```

**Explanation:**
- Prevents architectural violations (e.g., frontend importing backend-specific code)
- `sourceTag`: Tags applied to the consuming project
- `onlyDependOnLibsWithTags`: Allowed dependencies
- Ensures layered architecture (UI → Feature → Data → Util)

### 42.2.4 Nx with TypeScript Path Mapping

Nx automatically manages TypeScript path aliases for clean imports.

**Generated tsconfig.base.json:**

```typescript
{
  "compileOnSave": false,
  "compilerOptions": {
    "rootDir": ".",
    "sourceMap": true,
    "declaration": false,
    "moduleResolution": "node",
    "emitDecoratorMetadata": true,
    "experimentalDecorators": true,
    "importHelpers": true,
    "target": "es2015",
    "module": "esnext",
    "lib": ["es2020", "dom"],
    "skipLibCheck": true,
    "skipDefaultLibCheck": true,
    "baseUrl": ".",
    "paths": {
      "@myorg/shared-types": ["packages/shared-types/src/index.ts"],
      "@myorg/api-client": ["packages/api-client/src/index.ts"],
      "@myorg/ui-components": ["packages/ui-components/src/index.ts"]
    }
  },
  "exclude": ["node_modules", "tmp"]
}
```

**Usage in Code:**

```typescript
// apps/web/src/main.ts
import { User } from '@myorg/shared-types';  // Resolves to packages/shared-types
import { ApiClient } from '@myorg/api-client';
import { Button } from '@myorg/ui-components';

const user: User = { id: 1, name: 'John' };
const client = new ApiClient();
```

---

## 42.3 Turborepo

Turborepo is a high-performance build system for JavaScript/TypeScript monorepos, developed by Vercel. It excels at task orchestration and remote caching, making it ideal for CI/CD optimization.

### 42.3.1 Turborepo Configuration

**turbo.json Configuration:**

```typescript
{
  "$schema": "https://turbo.build/schema.json",
  "globalDependencies": ["**/.env.*local", ".env"],
  "pipeline": {
    "build": {
      "dependsOn": ["^build"],
      "outputs": [".next/**", "!.next/cache/**", "dist/**"],
      "env": ["API_URL", "DATABASE_URL"]
    },
    "test": {
      "dependsOn": ["build"],
      "outputs": ["coverage/**"],
      "inputs": ["src/**/*.ts", "src/**/*.tsx", "**/*.test.ts"]
    },
    "lint": {},
    "dev": {
      "cache": false,
      "persistent": true
    },
    "typecheck": {
      "dependsOn": ["^build"],
      "outputs": []
    }
  }
}
```

**Explanation:**
- `pipeline`: Defines task dependencies and caching rules
- `dependsOn`: `^build` means "build all dependencies first"
- `outputs`: Files to cache (hash-based storage)
- `env`: Environment variables that affect the cache key
- `cache: false`: Never cache (for dev servers)
- `persistent`: Task runs indefinitely (dev servers)

### 42.3.2 Task Orchestration and Caching

Turborepo analyzes the task graph and runs tasks in parallel when safe.

**Running Tasks:**

```typescript
// Run build across all packages
// npx turbo run build

// Run multiple tasks
// npx turbo run build test lint

// Filter specific packages
// npx turbo run build --filter=web-app
// npx turbo run build --filter=@myorg/api-client

// Filter by changes since git ref
// npx turbo run test --filter=[HEAD~10]

// Force skip cache
// npx turbo run build --force
```

**Workspace Configuration (package.json):**

```typescript
// Root package.json
{
  "name": "my-turborepo",
  "private": true,
  "scripts": {
    "build": "turbo run build",
    "dev": "turbo run dev --parallel",
    "lint": "turbo run lint",
    "test": "turbo run test",
    "typecheck": "turbo run typecheck",
    "clean": "turbo run clean && rm -rf node_modules"
  },
  "devDependencies": {
    "turbo": "^1.11.0",
    "typescript": "^5.3.0"
  }
}

// Individual package package.json
{
  "name": "@myorg/shared-types",
  "version": "1.0.0",
  "main": "./dist/index.js",
  "types": "./dist/index.d.ts",
  "scripts": {
    "build": "tsc",
    "clean": "rm -rf dist",
    "typecheck": "tsc --noEmit"
  }
}
```

### 42.3.3 Remote Caching

Turborepo's standout feature is remote caching, sharing build artifacts across team members and CI/CD.

**Setting Up Remote Cache:**

```typescript
// Link repository to Vercel Remote Cache
// npx turbo login
// npx turbo link

// Environment variables for CI
TURBO_TOKEN=your_token_here
TURBO_TEAM=your_team_slug

// GitHub Actions example
// .github/workflows/ci.yml
name: CI
on: [push]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-node@v3
      - uses: pnpm/action-setup@v2
      
      - name: Install dependencies
        run: pnpm install
        
      - name: Build
        run: pnpm turbo run build
        env:
          TURBO_TOKEN: ${{ secrets.TURBO_TOKEN }}
          TURBO_TEAM: ${{ vars.TURBO_TEAM }}
```

**Explanation:**
- Local cache: `node_modules/.cache/turbo`
- Remote cache: Vercel-hosted or self-hosted (Enterprise)
- Cache hits skip expensive operations (compilation, testing)
- Team members download pre-built artifacts

---

## 42.4 Lerna

Lerna is a tool for managing JavaScript/TypeScript monorepos, historically focused on versioning and publishing. While modern tools (Nx, Turborepo) handle builds, Lerna remains valuable for package publishing workflows.

### 42.4.1 Lerna with Modern Tools

Modern setups use Lerna for versioning/publishing alongside Nx or Turborepo for builds.

**lerna.json Configuration:**

```typescript
{
  "version": "independent",
  "npmClient": "pnpm",
  "command": {
    "publish": {
      "conventionalCommits": true,
      "message": "chore(release): publish",
      "registry": "https://npm.pkg.github.com"
    },
    "version": {
      "allowBranch": ["main", "release/*"],
      "ignoreChanges": ["*.md", "*.test.ts", "**/test/**"]
    }
  },
  "packages": ["packages/*", "apps/*"]
}
```

**Explanation:**
- `version: "independent"`: Each package versions independently vs. fixed (unified version)
- `conventionalCommits`: Automates versioning based on commit messages (feat:, fix:, etc.)
- `ignoreChanges`: Files that don't trigger version bumps

### 42.4.2 Versioning and Publishing Workflow

**Publishing Script:**

```typescript
// package.json
{
  "scripts": {
    "version": "lerna version",
    "publish": "lerna publish from-git",
    "publish:canary": "lerna publish --canary --preid beta"
  }
}

// Conventional commit examples:
// feat(api-client): add retry logic -> Minor version bump (1.0.0 -> 1.1.0)
// fix(shared-types): correct User interface -> Patch version bump (1.0.0 -> 1.0.1)
// feat(ui-components)!: remove deprecated props -> Major version bump (1.0.0 -> 2.0.0)
```

**GitHub Actions Release Workflow:**

```typescript
// .github/workflows/release.yml
name: Release
on:
  push:
    branches: [main]

jobs:
  release:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
        with:
          fetch-depth: 0  # Required for conventional commits
          
      - name: Setup Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '20'
          registry-url: 'https://npm.pkg.github.com'
          
      - name: Install dependencies
        run: pnpm install
        
      - name: Build packages
        run: pnpm turbo run build
        
      - name: Version and Publish
        run: |
          git config --global user.email "github-actions@github.com"
          git config --global user.name "GitHub Actions"
          npx lerna version --yes
          npx lerna publish from-git --yes
        env:
          NODE_AUTH_TOKEN: ${{ secrets.GITHUB_TOKEN }}
```

**Explanation:**
- `fetch-depth: 0`: Required to analyze commit history for conventional commits
- Lerna analyzes commits since last tag to determine version bumps
- Creates git tags and GitHub releases automatically
- Publishes to configured registry (npm, GitHub Packages, etc.)

---

## 42.5 Shared Type Packages

One of the primary benefits of TypeScript monorepos is sharing type definitions between frontend, backend, and mobile applications.

### 42.5.1 Designing Shared Type Packages

**Package Structure:**

```
packages/shared-types/
├── src/
│   ├── index.ts          # Public API exports
│   ├── models/           # Domain models
│   │   ├── user.ts
│   │   ├── order.ts
│   │   └── index.ts
│   ├── api/              # API contract types
│   │   ├── requests.ts
│   │   ├── responses.ts
│   │   └── errors.ts
│   ├── validation/       # Zod/io-ts schemas
│   │   └── schemas.ts
│   └── utils/            # Type utilities
│       └── helpers.ts
├── package.json
└── tsconfig.json
```

**Implementation:**

```typescript
// packages/shared-types/src/models/user.ts
export interface User {
  id: string;
  email: string;
  name: string;
  role: 'admin' | 'user' | 'guest';
  createdAt: Date;
  metadata?: Record<string, unknown>;
}

export type UserCreateInput = Omit<User, 'id' | 'createdAt'>;
export type UserUpdateInput = Partial<UserCreateInput>;

// Branded types for type-safe IDs
export type UserId = string & { __brand: 'UserId' };
export function createUserId(id: string): UserId {
  return id as UserId;
}

// packages/shared-types/src/api/requests.ts
export interface CreateUserRequest {
  email: string;
  name: string;
  role?: 'user' | 'guest';
}

export interface UpdateUserRequest {
  name?: string;
  role?: 'admin' | 'user' | 'guest';
}

// packages/shared-types/src/api/responses.ts
export interface ApiResponse<T> {
  data: T;
  meta?: {
    page?: number;
    limit?: number;
    total?: number;
  };
}

export interface ApiError {
  code: string;
  message: string;
  details?: Record<string, string[]>;
}

export type UserResponse = ApiResponse<User>;
export type UsersListResponse = ApiResponse<User[]>;

// packages/shared-types/src/index.ts
// Public API - only export what's intended for consumers
export * from './models/user';
export * from './models/order';
export * from './api/requests';
export * from './api/responses';
export * from './api/errors';

// Do not export internal utilities unless needed
// export * from './validation/schemas'; // Keep internal
```

### 42.5.2 Runtime Validation with Shared Types

Shared types should include runtime validation to ensure type safety at boundaries (API inputs, external data).

**Zod Integration:**

```typescript
// packages/shared-types/src/validation/schemas.ts
import { z } from 'zod';

export const UserSchema = z.object({
  id: z.string().uuid(),
  email: z.string().email(),
  name: z.string().min(1).max(100),
  role: z.enum(['admin', 'user', 'guest']),
  createdAt: z.coerce.date(),
  metadata: z.record(z.unknown()).optional()
});

export const CreateUserSchema = UserSchema.omit({
  id: true,
  createdAt: true
}).extend({
  role: z.enum(['user', 'guest']).optional()
});

// Infer TypeScript types from Zod schemas
export type User = z.infer<typeof UserSchema>;
export type CreateUserInput = z.infer<typeof CreateUserSchema>;

// Validation helpers
export function validateUser(data: unknown): User {
  return UserSchema.parse(data);
}

export function validateCreateUser(data: unknown): CreateUserInput {
  return CreateUserSchema.parse(data);
}

// Safe parsing (returns success/failure)
export function safeValidateUser(data: unknown) {
  return UserSchema.safeParse(data);
}
```

**Explanation:**
- Single source of truth: Zod schema defines both runtime validation and TypeScript types
- `z.infer<>` extracts TypeScript types from schemas
- Parse throws on invalid data; safeParse returns result object
- Coercion handles string dates from JSON APIs

### 42.5.3 Publishing Type Packages

**Package.json Configuration:**

```typescript
{
  "name": "@mycompany/shared-types",
  "version": "1.0.0",
  "description": "Shared TypeScript types for MyCompany applications",
  "main": "./dist/index.js",
  "types": "./dist/index.d.ts",
  "exports": {
    ".": {
      "types": "./dist/index.d.ts",
      "import": "./dist/index.mjs",
      "require": "./dist/index.js"
    },
    "./models": {
      "types": "./dist/models/index.d.ts",
      "import": "./dist/models/index.mjs",
      "require": "./dist/models/index.js"
    },
    "./validation": {
      "types": "./dist/validation/index.d.ts",
      "import": "./dist/validation/index.mjs",
      "require": "./dist/validation/index.js"
    }
  },
  "files": ["dist"],
  "scripts": {
    "build": "tsup src/index.ts --format cjs,esm --dts",
    "dev": "tsup src/index.ts --format cjs,esm --dts --watch",
    "test": "vitest run",
    "typecheck": "tsc --noEmit"
  },
  "dependencies": {
    "zod": "^3.22.0"
  },
  "devDependencies": {
    "tsup": "^8.0.0",
    "typescript": "^5.3.0"
  },
  "publishConfig": {
    "access": "restricted",
    "registry": "https://npm.pkg.github.com"
  }
}
```

**Explanation:**
- `exports`: Modern entry points supporting both ESM and CommonJS
- Subpath exports (`./models`, `./validation`) allow selective imports
- `tsup`: Zero-config bundler for TypeScript libraries (generates .js, .mjs, .d.ts)
- `files`: Only publish dist directory (not source)

---

## 42.6 API Contract Sharing

Maintaining type safety between frontend and backend requires sharing API contracts. Modern approaches include OpenAPI specifications, tRPC, and GraphQL.

### 42.6.1 OpenAPI (Swagger) Generation

Generate TypeScript types from OpenAPI specifications for end-to-end type safety.

**OpenAPI Generator Setup:**

```typescript
// openapi.config.ts
export default {
  input: './openapi.yaml',  // Backend API spec
  output: './packages/api-client/src/generated',
  clients: ['typescript-fetch'],  // or 'typescript-axios'
  options: {
    supportsES6: true,
    typescriptThreePlus: true,
    modelPropertyNaming: 'original',
    enumPropertyNaming: 'UPPERCASE'
  }
};

// Generated usage
import { UserApi, Configuration } from '@myorg/api-client/generated';

const config = new Configuration({ basePath: 'https://api.example.com' });
const userApi = new UserApi(config);

// Fully typed API calls
const response = await userApi.getUserById({ id: '123' });
// response.user is typed based on OpenAPI schema
```

**Contract-First Development:**

```typescript
// packages/api-contract/openapi.yaml
openapi: 3.0.0
info:
  title: My API
  version: 1.0.0
paths:
  /users/{id}:
    get:
      operationId: getUserById
      parameters:
        - name: id
          in: path
          required: true
          schema:
            type: string
            format: uuid
      responses:
        '200':
          description: User found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
        '404':
          $ref: '#/components/responses/NotFound'

components:
  schemas:
    User:
      type: object
      required: [id, email, name]
      properties:
        id:
          type: string
          format: uuid
        email:
          type: string
          format: email
        name:
          type: string
        role:
          type: string
          enum: [admin, user, guest]
```

### 42.6.2 tRPC for Type-Safe APIs

tRPC provides end-to-end typesafe APIs without code generation by sharing the router definition.

**Router Definition (Shared):**

```typescript
// packages/api-router/src/router.ts
import { initTRPC } from '@trpc/server';
import { z } from 'zod';
import { UserSchema } from '@myorg/shared-types';

const t = initTRPC.create();

export const appRouter = t.router({
  user: t.router({
    getById: t.procedure
      .input(z.object({ id: z.string().uuid() }))
      .output(UserSchema)
      .query(async ({ input }) => {
        // Implementation in backend
        return await db.user.findById(input.id);
      }),
      
    create: t.procedure
      .input(z.object({
        email: z.string().email(),
        name: z.string(),
        role: z.enum(['user', 'guest']).optional()
      }))
      .mutation(async ({ input }) => {
        return await db.user.create(input);
      })
  })
});

// Export type definition
export type AppRouter = typeof appRouter;
```

**Backend Integration:**

```typescript
// apps/api/src/server.ts
import { createHTTPServer } from '@trpc/server/adapters/standalone';
import { appRouter } from '@myorg/api-router';

const server = createHTTPServer({
  router: appRouter,
});

server.listen(3000);
```

**Frontend Integration:**

```typescript
// apps/web/src/trpc.ts
import { createTRPCReact } from '@trpc/react-query';
import type { AppRouter } from '@myorg/api-router';

export const trpc = createTRPCReact<AppRouter>();

// Usage in component
function UserProfile({ userId }: { userId: string }) {
  const { data, error } = trpc.user.getById.useQuery({ id: userId });
  
  // data is fully typed as User
  // Error types are inferred
  
  if (error) return <div>Error: {error.message}</div>;
  if (!data) return <div>Loading...</div>;
  
  return (
    <div>
      <h1>{data.name}</h1>
      <p>{data.email}</p>
    </div>
  );
}
```

**Explanation:**
- `AppRouter` type is shared between frontend and backend
- No code generation needed; types flow through the procedure chain
- Input/output schemas validated at runtime (Zod) and compile-time (TypeScript)
- Auto-completion and refactoring work across the full stack

### 42.6.3 GraphQL with Code Generation

For GraphQL APIs, use GraphQL Code Generator to create TypeScript types from schemas.

**Configuration:**

```typescript
// codegen.ts
import type { CodegenConfig } from '@graphql-codegen/cli';

const config: CodegenConfig = {
  schema: './apps/api/src/schema.graphql',
  documents: ['./apps/web/src/**/*.graphql', './apps/web/src/**/*.{ts,tsx}'],
  generates: {
    './packages/graphql-types/src/generated.ts': {
      plugins: [
        'typescript',
        'typescript-operations',
        'typescript-react-apollo'
      ],
      config: {
        withHooks: true,
        withComponent: false,
        withHOC: false,
        skipTypename: false,
        enumsAsTypes: true,
        scalars: {
          DateTime: 'string',
          JSON: 'Record<string, any>'
        }
      }
    }
  }
};

export default config;
```

**Usage:**

```typescript
// Generated types provide:
// - TypeScript types for all GraphQL types
// - Typed React hooks (useGetUserQuery, useCreateUserMutation)
// - Variables and response types

import { useGetUserQuery } from '@myorg/graphql-types';

function UserComponent({ id }: { id: string }) {
  const { data, loading } = useGetUserQuery({
    variables: { id }  // Type-checked against schema
  });
  
  // data.user is fully typed
  return <div>{data?.user.name}</div>;
}
```

---

## 42.7 Chapter Summary and Exercises

### Chapter Summary

This chapter covered scaling TypeScript across large organizations:

1. **Monorepo Structure**: Using npm/Yarn/pnpm workspaces to manage multiple packages with shared dependencies and TypeScript project references for incremental builds.

2. **Nx**: Comprehensive monorepo tooling with intelligent caching, module boundary enforcement, and code generation. Best for enterprise teams needing strict architectural constraints.

3. **Turborepo**: High-performance task runner with remote caching capabilities. Ideal for Vercel deployments and teams prioritizing build speed.

4. **Lerna**: Legacy tool still valuable for versioning and publishing workflows using conventional commits.

5. **Shared Type Packages**: Centralizing domain models, API contracts, and validation schemas (Zod) to ensure consistency across frontend and backend.

6. **API Contract Sharing**: End-to-end type safety through OpenAPI generation, tRPC (typesafe procedures), and GraphQL code generation.

### Industry Standards Checklist

```
┌─────────────────────────────────────────────────────────────────────┐
│                    TypeScript at Scale Checklist                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Repository Setup                                                  │
│   □ Use pnpm workspaces for efficient disk usage                    │
│   □ Configure TypeScript project references for incremental builds  │
│   □ Implement consistent tsconfig inheritance (base → packages)     │
│                                                                     │
│   Build & CI/CD                                                     │
│   □ Implement remote caching (Turborepo or Nx)                      │
│   □ Parallelize independent tasks in CI                             │
│   □ Use affected commands to minimize CI time                       │
│                                                                     │
│   Code Quality                                                      │
│   □ Enforce module boundaries with tags/constraints                 │
│   □ Shared linting configuration across all packages                │
│   □ Automated testing at package boundaries                         │
│                                                                     │
│   Type Safety                                                       │
│   □ Runtime validation (Zod/io-ts) paired with TypeScript types     │
│   □ API contract sharing (tRPC or OpenAPI)                          │
│   □ Branded types for ID safety                                     │
│                                                                     │
│   Publishing                                                        │
│   □ Conventional commits for automated versioning                   │
│   □ Scoped packages (@org/package)                                  │
│   □ Proper ESM/CJS dual exports                                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Practical Exercises

**Exercise 1: Workspace Configuration**

Set up a monorepo with three packages:
1. `@myorg/types` - Shared TypeScript interfaces
2. `@myorg/utils` - Utility functions (depends on types)
3. `@myorg/api` - Express API using both packages

Configure TypeScript project references and ensure that changes to `types` trigger rebuilds of dependent packages.

**Exercise 2: Build System Comparison**

Configure both Nx and Turborepo for the same monorepo structure:
- Create equivalent task pipelines
- Configure module boundary rules (Nx) or workspace constraints
- Benchmark build times with cold cache vs. warm cache
- Document when to choose each tool

**Exercise 3: End-to-End Type Safety**

Implement a full-stack feature using tRPC:
1. Create a shared router package with user CRUD operations
2. Implement the backend in an Express app
3. Consume the API in a React frontend with full type inference
4. Add Zod validation schemas to the shared package
5. Ensure that changing the router breaks the frontend build if types mismatch

**Exercise 4: Publishing Workflow**

Set up automated publishing:
1. Configure Lerna with conventional commits
2. Create a GitHub Action that:
   - Runs tests on PRs
   - Builds affected packages on merge to main
   - Versions and publishes packages automatically
3. Implement pre-release (canary) publishing for beta features

**Exercise 5: API Contract Migration**

Migrate an existing REST API to use OpenAPI:
1. Write OpenAPI 3.0 specification for a user management API
2. Generate TypeScript client using openapi-generator
3. Replace manual fetch calls with generated client in frontend
4. Set up CI check to ensure backend implements spec correctly
5. Configure breaking change detection in CI

---

## Next Chapter Preview

### Chapter 43: TypeScript Best Practices

In the next chapter, we will consolidate our knowledge into actionable guidelines for professional TypeScript development:

- **Code Organization**: File structure, barrel exports, and module design patterns
- **Naming Conventions**: PascalCase vs camelCase, interface naming (I-prefix or not), type vs interface decisions
- **Avoiding Any**: Strict type enforcement strategies and gradual migration paths
- **Error Handling**: Typed errors, Result types, and exception handling patterns
- **Documentation**: JSDoc/TSDoc standards for public APIs and IntelliSense optimization
- **Performance**: Type-only imports, const assertions, and bundle size optimization techniques

This chapter will serve as your style guide and reference for maintaining high-quality TypeScript codebases in production environments.

---